In [1]:
import imp
import Data
import utils
import Predictors, Road_Graph, Neural_Network
imp.reload(Data)
imp.reload(Neural_Network)
imp.reload(utils)
imp.reload(Predictors)
imp.reload(Road_Graph)

from Road_Graph import *
from Predictors import *
from Data import *
from utils import *
import matplotlib.pyplot as plt
import numpy as np
from torch.autograd import Variable
import torch
from torch import nn, autograd
import torch.nn.functional as F
import pickle

In this part, we focus on a simple road map (rep as a graph below), and generate random path on it.
<img src="../img/naive_road.png" alt="Drawing" style="width: 400px;"/>

In [2]:
graph_dict = {}
graph_dict = graph_dict.fromkeys(range(24))
edges = [(i, i+1) for i in range(5)] \
        + [(i+6, i+7) for i in range(5)]\
                        +[(i+12, i+13) for i in range(5)]\
                     + [(i+18, i+19) for i in range(5)]\
                     + [(0,6),(6,12), (12,18)]\
                     + [(pair[0]+1, pair[1]+1) for pair in [(0,6),(6,12), (12,18)]]\
                     + [(pair[0]+2, pair[1]+2) for pair in [(0,6),(6,12), (12,18)]]\
                     + [(pair[0]+3, pair[1]+3) for pair in [(0,6),(6,12), (12,18)]]\
                     + [(pair[0]+4, pair[1]+4) for pair in [(0,6),(6,12), (12,18)]]\
                     + [(pair[0]+5, pair[1]+5) for pair in [(0,6),(6,12), (12,18)]]
for i,j in edges:
    if not graph_dict[i]:
        graph_dict[i] = [j]
    else:
        graph_dict[i].append(j)
    if not graph_dict[j]:
        graph_dict[j] = [i]
    else:
        graph_dict[j].append(i)

graph = Road_Graph(graph_dict)

# Markov Data

In [13]:
def path_to_training_pair2(path):
    return path[1:], path[:-1]

def get_prob(x):
    nb = graph.neighbors(x)
    W_list = [W[j].view(-1,1) for j in nb]
    Ws = torch.cat(W_list, dim = 1)
    W_next = model(W[x])
    #print('W_next:', W_next)
    #print(W_next.size(), Ws.size())
    W_next_rep = W_next.view(-1,1).repeat(1,Ws.size()[1])
    #print(W_next_rep.size(), Ws.size())
    logit = torch.sum(W_next_rep*Ws, 0)
    prob = F.softmax(logit, dim = 0)
    return prob

def get_loss(x, y):
    nb = graph.neighbors(x)
    #import pdb; pdb.set_trace()
    W_list = [W[j].view(-1,1) for j in nb]
    Ws = torch.cat(W_list, dim = 1)
    W_next = model(W[x])
    #print('W_next:', W_next)
    #print(W_next.size(), Ws.size())
    W_next_rep = W_next.view(-1,1).repeat(1,Ws.size()[1])
    #print(W_next_rep.size(), Ws.size())
    logit = torch.sum(W_next_rep*Ws, 0)
    prob = F.softmax(logit, dim = 0)
    loss = -torch.log(prob[np.where(nb == y)])
    return loss

In [ ]:
dtype = torch.FloatTensor

data = random_walk_data(graph,10000)

im_D = 5
N = 24
h_D = 100


loss_list = []
W = Variable(torch.randn(N,im_D).type(dtype), requires_grad=True)
model = torch.nn.Sequential(
        torch.nn.Linear(im_D, h_D),
        torch.nn.ReLU(),
        torch.nn.Linear(h_D, im_D),
    )
for k in range(len(data)):
    path = data[k]
    x,y = path_to_training_pair2(path)
    
    optimizer = torch.optim.Adam(list(model.parameters()) + [W], lr=0.001) # remove [W] for a fixed random embeding
    for i in range(len(x)):
        loss = get_loss(x[i], y[i])
        loss_list.append(loss)
        
    if k%50 == 0:
        batch_loss = torch.sum(torch.cat(loss_list)) / len(loss_list)
        optimizer.zero_grad()
        batch_loss.backward()
        optimizer.step()
        loss_list = []
        print(batch_loss.data.numpy())
 

In [16]:
def opt_log_loss():
    s = 0
    for i in range(24):
        n = len(get_prob(i)) - 1
        s += -np.sum(np.ones(n) /n * np.log(np.ones(n)/n))
    return s/24
opt_log_loss()

NameError: name 'W' is not defined

In [ ]:
def opt_log_loss():
    s = 0
    for i in range(24):
        p_ = get_prob(i).data.numpy()
        n = len(get_prob(i))
        s += -np.sum(np.ones(n) /n * np.log(np.ones(n)/n))
    return s/24
opt_log_loss()

In [ ]:
def log_loss():
    s = 0
    for i in range(24):
        p_ = get_prob(i).data.numpy()
        n = len(get_prob(i))
        s += -np.sum(np.ones(n) /n * np.log(p_))
    return s/24
log_loss()

In [ ]:
def log_loss():
    s = 0
    for i in range(24):
        p_ = get_prob(i).data.numpy()
        n = len(get_prob(i))
        s += -np.sum(np.ones(n) /n * np.log(p_))
    return s/24
log_loss()

In [ ]:
get_prob(1)

# Hist Dependent Data

In [ ]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.gru = nn.GRU(input_size, hidden_size, n_layers)
        self.lin = nn.Linear(hidden_size, output_size)
    
    def forward(self, inp, hidden):
        output, hidden = self.gru(inp.view(1, 1, self.input_size), hidden)
        output = self.lin(output) 
        return output, hidden        

    def init_hidden(self):
        return Variable(torch.zeros(self.n_layers, 1, self.hidden_size))

In [ ]:
def path_loss(path):
    h = model.init_hidden()
    output_list = []
    for i in path:
        o, h = model(W[i], h)
        output_list.append(o)
    loss_list = []
    for i in range(len(path) - 1):
        x = path[i]
        y = path[i+1]
        nb = graph.neighbors(x)
        W_list = [W[j].view(-1,1) for j in nb]
        Ws = torch.cat(W_list, dim = 1)
        W_next = output_list[i]
        W_next_rep = W_next.view(-1,1).repeat(1,Ws.size()[1])
        logit = torch.sum(W_next_rep*Ws, 0)
        prob = F.softmax(logit, dim = 0)
        loss_list.append(-torch.log(prob[np.where(nb == y)]))
    return torch.sum(torch.cat(loss_list))

def get_prob(path):
    h = model.init_hidden()
    for i in path:
        o, h = model(W[i], h)
    x = path[-1]
    nb = graph.neighbors(x)
    W_list = [W[j].view(-1,1) for j in nb]
    Ws = torch.cat(W_list, dim = 1)
    W_next = o
    W_next_rep = W_next.view(-1,1).repeat(1,Ws.size()[1])
    logit = torch.sum(W_next_rep*Ws, 0)
    prob = F.softmax(logit, dim = 0)
    return prob

In [ ]:
data = random_walk_data(graph, 10000, go_back=False)

In [ ]:
im_D = 50
N = 24
h_D = 100
dtype = torch.FloatTensor


W = Variable(torch.randn(N,im_D).type(dtype), requires_grad=True)
model = RNN(im_D, h_D, im_D)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # remove [W] for a fixed random embeding

loss_list = []
s = 0
for k in range(len(data)):
    path = data[k]
    loss = path_loss(path)
    loss_list.append(loss)    
    s += len(path)
    if k%50 == 0:
        batch_loss = torch.sum(torch.cat(loss_list)) / s
        s = 0
        optimizer.zero_grad()
        batch_loss.backward()
        optimizer.step()
        loss_list = []
        print(batch_loss.data.numpy())
 

In [ ]:
get_prob([1,2,3,4])

# Modulized Code

In [7]:
data = random_walk_data(graph, 20000, go_back=False)

In [8]:
rnn_pd = RNN_Predictor(graph)

In [9]:
rnn_pd.train(data)

[ 1.49057841]
[ 1.31368899]
[ 1.21806753]
[ 1.20786726]
[ 1.16426241]
[ 1.11113167]
[ 1.08402061]
[ 1.02662027]
[ 1.08754182]
[ 1.03246641]
[ 1.02397311]
[ 0.97215527]
[ 0.9794023]
[ 0.99841374]
[ 0.95839667]
[ 0.93975204]
[ 0.93028617]
[ 0.9550643]
[ 0.93516916]
[ 0.92080152]
[ 0.91433632]
[ 0.90261841]
[ 0.8887884]
[ 0.89584726]
[ 0.88376725]
[ 0.87292325]
[ 0.90974182]
[ 0.84714019]
[ 0.86564213]
[ 0.91250235]
[ 0.8756271]
[ 0.87300533]
[ 0.83986932]
[ 0.86106211]
[ 0.85313326]
[ 0.83331144]
[ 0.84302658]
[ 0.79348326]
[ 0.84175235]
[ 0.81325763]
[ 0.82886684]
[ 0.79469347]
[ 0.82364309]
[ 0.83853394]
[ 0.80734676]
[ 0.83412749]
[ 0.8479777]
[ 0.84218377]
[ 0.83010334]
[ 0.849352]
[ 0.8261224]
[ 0.8209154]
[ 0.83725631]
[ 0.84606367]
[ 0.77279347]
[ 0.81535184]
[ 0.83273989]
[ 0.82602894]
[ 0.82573992]
[ 0.8462832]
[ 0.79583359]
[ 0.81302643]
[ 0.83354908]
[ 0.80464822]
[ 0.82740986]
[ 0.79632878]
[ 0.80040205]
[ 0.81691951]
[ 0.82699639]
[ 0.81366253]
[ 0.80883461]
[ 0.81112063]
[ 

In [10]:
rnn_pd.predict([0,1,2,3])

(array([2, 4, 9]), Variable containing:
  0.0126
  0.6001
  0.3873
 [torch.FloatTensor of size 3])